In [1]:
import twitter
import os
import importlib 
import pandas as pd
import numpy as np

# from model import *
from twitter.scraper import *

test_search_people = lambda: search_people(query='rocket lab')
test_get_user_tweets = lambda: get_user_tweets(user_id='91145174', minimum_tweets=100)
test_get_comments = lambda: get_comments(tweet_id='1958169670541771037')
test_search_tweets = lambda: search_tweets(query='rocket lab')
test_all = lambda: test_search_people() + test_get_user_tweets() + test_get_comments() + test_search_tweets()

def test(user_id: str = None, tweet_id: str = None, query: str = None):
    rocket_lab = '91145174'
    peter_beck = '976574172468936704'
    try:
        tweets, cursor_bottom = get_user_tweets(user_id if user_id else rocket_lab, minimum_tweets=100)
        print("Account tweet extraction successful: ", len(tweets))
    except Exception as e:
        print(e)
    
    try:
        # 1952346711625139200 comment error
        tweets, cursor_bottom = get_comments(tweet_id if tweet_id else '1958169670541771037')
        print(tweets_to_pipe_table(tweets))
        print("Comments extraction successful: ", len(tweets))
    except Exception as e:
        print(e)
    try:
        users, cursor_bottom = search_people(query if query else 'elon musk')
        print("People search successful: ", len(users))
    except Exception as e:
        print(e)
   
    try:
        tweets, cursor_bottom = search_tweets(query if query else 'rocket lab')
        print("Tweets search successful: ", len(tweets))
    except Exception as e:
        print(e)
   
pd.set_option("display.max_colwidth", None)  # show full text in each cell


In [2]:
importlib.reload(twitter)
from twitter import *
rocket_lab = '91145174'
peter_beck = '976574172468936704'
nvidia= '61559439'

tweets = get_user_tweets(nvidia, period='month=1')[0]

tweet_with_most_comments = max(tweets, key=lambda x: x.replies)
top_20_tweets = sorted(tweets, key=lambda x: x.replies, reverse=True)[:20]
comments = get_comments(tweet_with_most_comments.tweet_id, minimum=200)[0]
original_tweet = comments[0]
comments = comments[1:]


TweetTombstone
TweetTombstone


In [3]:
cleaned_tweets = []
def stringify_tweet(tweet: Tweet):
    full_tweet = []
    cleaned_tweet_text = clean_tweet(tweet.text)
    if cleaned_tweet_text: 
        full_tweet.append(cleaned_tweet_text)
    if tweet.post_image_description:
        full_tweet.append(tweet.post_image_description)
    if tweet.post_video_description:
        full_tweet.append(tweet.post_video_description)
    
    if tweet.quoted_tweet:
        full_tweet+= [stringify_tweet(tweet.quoted_tweet)]
    return "\n".join(full_tweet)

print(len(tweets))
for tweet in tweets:
    if tweet.retweeted_tweet: continue
    full_tweet = stringify_tweet(tweet)
    if not full_tweet: continue
    cleaned_tweets.append(full_tweet)
    # print(tweet.created_at)
    print(full_tweet)
    print("-"*100)
print("Newest Tweet Date: ", tweets[0].created_at)
print("Oldest Tweet Date: ", tweets[-1].created_at)
print(f"{len(cleaned_tweets)}/{len(tweets)}")

58
💬 “We're trying to make the most open approach to AI development that the world has ever seen.” 💡 Openness isn’t optional — it’s essential for the future of AI. That’s why Nemotron was built: to set new standards for speed, transparency, and versatility in enterprise AI.
Bryan Catanzaro stands in a modern office, wearing glasses, a light shirt, and a dark blazer, gesturing while speaking. Multiple people collaborate at computer workstations, viewing screens displaying technical data and charts in a high-tech environment with server racks. A webpage from Hugging Face shows a dark-themed interface with model listings and technical documentation. A bar chart compares AI model performance, featuring labeled columns and percentages. Another scene shows Bryan Catanzaro in a sleek, futuristic setting, speaking animatedly with hand gestures.
----------------------------------------------------------------------------------------------------
NVIDIA is live at #ClimateWeekNYC 🌍 ⚡ From AI for 

In [4]:
cleaned_comments = []

for comment in comments:
    cleaned_comment = stringify_tweet(comment)
    if not cleaned_comment: continue
    cleaned_comments.append(cleaned_comment)
    print(cleaned_comment)
    print("-"*100)

print("Oldest Comment Date: ", comments[0].created_at)
print("Newest Comment Date: ", comments[-1].created_at)
print(f"{len(cleaned_comments)}/{len(comments)}")



Read More:
----------------------------------------------------------------------------------------------------
@OpenAI @grok how many blackwell GPUs would this come up to?
----------------------------------------------------------------------------------------------------
@OpenAI How much is that as a percentage of current us energy consumption?
----------------------------------------------------------------------------------------------------
@OpenAI Shame on you for selling 32 gb 5090 from 4000 usd Cant wait the day China will shatter your monopoly
----------------------------------------------------------------------------------------------------
@OpenAI So Jensen pays Sam $100B for a stake in OpenAI and then Sam takes that $100B and buys GPUs from Jensen. AI bubble loading
----------------------------------------------------------------------------------------------------
@OpenAI Scam Altman is never to be trusted @nvdia Don't do it Pull out
--------------------------------------

In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
vader_df = pd.DataFrame()
vader_df["comment"] = cleaned_comments
vader_sentiments = [analyzer.polarity_scores(sentence) for sentence in cleaned_comments]
# vader_df["neg"] = [sentiment["neg"] for sentiment in vader_sentiments]
# vader_df["neu"] = [sentiment["neu"] for sentiment in vader_sentiments]
# vader_df["pos"] = [sentiment["pos"] for sentiment in vader_sentiments]
# vader_df["compound"] = [sentiment["compound"] for sentiment in vader_sentiments]
vader_df["overall"] = [["❌", "😐", "✅"][np.argmax([sentiment["neg"], sentiment["neu"], sentiment["pos"]])] for sentiment in vader_sentiments]
vader_df["confidence"] = [sentiment[["neg", "neu", "pos"][np.argmax([sentiment["neg"], sentiment["neu"], sentiment["pos"]])]] for sentiment in vader_sentiments]
print(vader_df.groupby("overall").size())
vader_df

overall
✅     10
❌      8
😐    213
dtype: int64


,comment,overall,confidence
0,Read More:,😐,1.000
1,@OpenAI @grok how many blackwell GPUs would this come up to?,😐,1.000
2,@OpenAI How much is that as a percentage of current us energy consumption?,😐,0.851
3,@OpenAI Shame on you for selling 32 gb 5090 from 4000 usd Cant wait the day China will shatter your monopoly,😐,0.866
4,@OpenAI So Jensen pays Sam $100B for a stake in OpenAI and then Sam takes that $100B and buys GPUs from Jensen. AI bubble loading,😐,1.000
...,...,...,...
226,@OpenAI What was the more important consideration when making the partnership...the power (access to electricity) or the programing (access to the software)? Surely it can't just be about hardware and being able to build data centers and supercomputers and having internet connections.,😐,0.889
227,@OpenAI NVIDIA和OpenAI这波合作直接给AI基建加满油啊！10 gigawatts的GPU供应量太夸张了，明天$NVDA股价不飞都难。不过这种gigascale项目周期长，短期利好但长期还得看实际落地。,😐,1.000
228,@OpenAI Time for a deal with $GLXY,😐,1.000
229,"@OpenAI Yo, this AI storm’s gonna hit crypto soon, and I’m dead sure projects with low mc and high potential are about to take off. Among them, $IMGN is my top pick for investing. Just do a bit of digging, and you’ll get what I’m saying. Just imagine and create with @imgn_ai",😐,0.789


In [7]:
import emoji
import re, pandas as pd, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, OpenAI
from sentence_transformers import SentenceTransformer
import openai

roberta_model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=roberta_model_path, tokenizer=roberta_model_path)
roberta_sentiments= sentiment_task(cleaned_comments)
roberta_df = pd.DataFrame()
roberta_df["comment"] = cleaned_comments
labels = {"negative": "❌", "neutral": "😐", "positive": "✅"}
roberta_df["sentiment"] = [labels[sentiment["label"]] for sentiment in roberta_sentiments]
roberta_df["confidence"] = [sentiment["score"] for sentiment in roberta_sentiments]

c:\Users\Admin\Documents\playground\twitter_sentiment_langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification m

In [18]:
print(roberta_df.groupby("sentiment").size())
roberta_df[roberta_df["sentiment"] == "❌"]
roberta_df[roberta_df["sentiment"] == "✅"]
roberta_df[roberta_df["sentiment"] == "😐"]
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    # print(roberta_df)
from IPython.display import display, HTML
display(HTML(roberta_df.to_html(index=False)))


sentiment
✅     82
❌     45
😐    104
dtype: int64


comment,sentiment,confidence
Read More:,😐,0.761533
@OpenAI @grok how many blackwell GPUs would this come up to?,😐,0.903998
@OpenAI How much is that as a percentage of current us energy consumption?,😐,0.896226
@OpenAI Shame on you for selling 32 gb 5090 from 4000 usd Cant wait the day China will shatter your monopoly,❌,0.924054
@OpenAI So Jensen pays Sam $100B for a stake in OpenAI and then Sam takes that $100B and buys GPUs from Jensen. AI bubble loading,😐,0.846367
@OpenAI Scam Altman is never to be trusted @nvdia Don't do it Pull out,❌,0.928154
"@OpenAI\nA detailed black and white illustration of a snake forming an infinity symbol, with its head biting its tail. The snake has textured scales and a coiled body creating two loops.",😐,0.918206
"@OpenAI How is that legal? Paying a customer to buy your products, isn't that money laundering?",❌,0.638372
"@OpenAI OpenAI: ""I need money for GPUs!"" NVIDIA: ""Here are $100B"" *Stock market value of both skyrockets* OpenAI: ""I want to purchase $100B worth of GPUs. Thanks for the money, btw"" NVIDIA: ""No problem bro. Want some more?"" Infinite money machine",✅,0.769856
@OpenAI Now the dollar feels it's backed by GPUs. Call it the cuda standard,😐,0.720985


In [ ]:
client = openai.OpenAI()
representation_model = OpenAI(client, model="gpt-4o", chat=True)
openai_BertTopic_model = BERTopic(representation_model=representation_model)

representation_model = KeyBERTInspired()
keybert_BertTopic_model = BERTopic(representation_model=representation_model)

In [ ]:
# BERTopic fitting
keybert_topics, keybert_probs = keybert_BertTopic_model.fit_transform(cleaned_tweets)
keybert_topic_info = keybert_BertTopic_model.get_topic_info()  # topic list with sizes
keybert_topic_info


openai_topics, openai_probs = openai_BertTopic_model.fit_transform(cleaned_tweets)
openai_topic_model = openai_BertTopic_model.reduce_topics(cleaned_tweets, nr_topics="auto")
openai_topic_info = openai_BertTopic_model.get_topic_info()  # topic list with sizes
openai_topic_info